# Analysis of quantitative experiment data

This notebook turns the event sequence collected by the Unity program and turns it into our quantitative metrics.

## Load Data

In [1]:
import numpy as np
import pandas as pd
import os

In [4]:
in_path = "ExperimentData/Manip/Control/"
out_path = "Metrics/Manip/Control"

In [5]:
dfs = {}
files = os.listdir(in_path)

for f in files:
    fname = f.split('/')[-1].split('.')[0]
    dfs[fname] = pd.read_csv(os.path.join(in_path, f))


## Extract Metrics

In [6]:
def extractMetrics(events):
    prev_time = events.iat[0, 4]
    times = []
    grabcounts = []
    grabcount = 0
    totalcompleted = 0
    for e in events.values:
        if e[0] == "GrabLeft" or e[0] == "GrabRight":
            grabcount += 1
        elif e[0] == "GoalReached":
            times.append(e[4] - prev_time)
            prev_time = e[4]
            grabcounts.append(grabcount)
            grabcount = 0
            totalcompleted += 1
    return pd.DataFrame({"times": times, "grabcounts": grabcounts, "completed": totalcompleted})

In [7]:
metrics = {}
for (k,v) in dfs.items():
    metrics[k] = extractMetrics(v)

## Write Results

In [8]:
os.makedirs(out_path, exist_ok=True)
for k in metrics.keys():
    metrics[k].to_csv(os.path.join(out_path, k + '.csv'), index=False)